In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import numpy as  np
from dataclasses import dataclass

@dataclass
class padainput:
    inputfile : str = "dipole.txt"
    outfile : str = "ftoutput.txt"
    limit : int = 10000
    frequency : float = 10.0
    fmin : float = 0.0
    f0 : str = "10;0;0"
    damping : str = "exp"
    gamma : np.float64 = 1.0e-3
    pad : int = 0
    rstep : int = 1
    resampling : bool = False
    diagn : bool = False
    pulseFmax : np.float64 = 1.0
    dw : float = 0.03
    preproc : bool = True 

wfilenamex = "./omp.mklparallel.quad.x/weighted_dip.txt"
wfilenamey = "./omp.mklparallel.quad.y/weighted_dip.txt"
wfilenamez = "./omp.mklparallel.quad.z/weighted_dip.txt"

dipolexfilename =  "./omp.mklparallel.quad.x/dipole.txt"
ouputxfilename = "alongx.txt"
dipoleyfilename =  "./omp.mklparallel.quad.y/dipole.txt"
ouputyfilename = "alongy.txt"
dipolezfilename =  "./omp.mklparallel.quad.z/dipole.txt"
ouputzfilename = "alongz.txt"

PADEDIR = "/home/redo/Sources/BERTHA/Exanalysis/misc/pade/"

usefiles = False

inputvals = padainput()
inputvals.gamma = 2.0e-6
inputvals.dw = 0.02 # 0.02
inputvals.damping = "gauss"
inputvals.fmin = 2.5 # 2.5 
inputvals.frequency = 6.5 # 6.5 
inputvals.limit = 25000 # 25000

sys.path.append(PADEDIR)
import pade_transform

spectra_values = {}

inputvals.inputfile = dipolexfilename
inputvals.outfile = ouputxfilename
spectra_values[ouputxfilename] = pade_transform.run_pade(inputvals)

inputvals.inputfile = dipoleyfilename
inputvals.outfile = ouputyfilename
spectra_values[ouputyfilename] = pade_transform.run_pade(inputvals)

inputvals.inputfile = dipolezfilename
inputvals.outfile = ouputzfilename
spectra_values[ouputzfilename] = pade_transform.run_pade(inputvals)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure

if usefiles:
    spectra_values = {}
    for fname in [ouputxfilename,  ouputyfilename, ouputzfilename]:
        fp = open(fname)
        spectra_values[fname] = []
        for l in fp:
            sl = l.split()
            spectra_values[fname].append(np.float64(sl))
        fp.close()
        npa = np.asarray(values[fname])
        spectra_values[fname] = npa
    
#print(values["alongx.tzt"])

x_total = []
y_total = []

for i, val in enumerate(spectra_values[ouputxfilename]):
    x_total.append(val[0]*27.2114)
    y_total.append(val[0]*(val[2]+spectra_values[ouputyfilename][i][2]+ \
                     spectra_values[ouputzfilename][i][2]))

figure(figsize=(12, 10), dpi=80)
plt.plot(x_total, y_total)

In [ ]:

w_dipole_values = {}
for fname in [wfilenamex, wfilenamey, wfilenamez]:
    fp = open(fname, "r")
    w_dipole_values[fname] = []
    for l in fp:
        w_dipole_values[fname].append(np.float64(l.split())) 

    w_dipole_values[fname] = np.asarray(w_dipole_values[fname])

    fp.close()

w_spectra_values = {}
for k in w_dipole_values:
    x = w_dipole_values[k][:,0]
    w_spectra_values[k] = []
    for ipair in range(1, w_dipole_values[k].shape[1]):
        print("%20s %3d of  %3d"%(k, ipair, w_dipole_values[k].shape[1]))
        y = w_dipole_values[k][:,ipair]
        inputa = np.column_stack((x,y))
        inputvals.outfile = "spectrum_"+k+"_"+str(ipair)+".txt"
        output = spectra_values[ouputxfilename] = pade_transform.run_pade(inputvals, \
                                            inputa=inputa, verbose=False)

        w_spectra_values[k].append(output)


In [ ]:
total_y = []
total_x = []
thefirst = True

for ipair in range(len(w_spectra_values[wfilenamex])):
    x = []
    y = []

    for i, val in enumerate(w_spectra_values[wfilenamex][ipair]):
        x.append(val[0]*27.2114)
        y.append(val[0]*(val[2]+w_spectra_values[wfilenamey][ipair][i][2]+ \
                     w_spectra_values[wfilenamez][ipair][i][2]))
        
    if thefirst:
        total_x= np.asarray(x)
        total_y = np.asarray(y)
        thefirst = False
    else:
        total_y += np.asarray(y)

    figure(figsize=(12, 10), dpi=80)
    plt.clf()
    plt.plot(x, y)

figure(figsize=(12, 10), dpi=80)
plt.clf()
plt.plot(total_x, total_y)
plt.plot(x_total, y_total)